# 0. Download the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!kaggle datasets download -d ryanmouton/ohiot1dm -p /content/drive/MyDrive/ddls_final_project/
!unzip /content/drive/MyDrive/ddls_final_project/ohiot1dm.zip -d /content/drive/MyDrive/ddls_final_project/data

# 1. Model training

In [ ]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Conv1D, MaxPooling1D, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
import joblib
import matplotlib.pyplot as plt
import os
import matplotlib.dates as mdates
from datetime import timedelta

## 1.1 Data Loading and Preprocessing

In [ ]:
def load_xml_data(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    events = []

    for event in root.findall('.//glucose_level/event'):
        timestamp = event.get('ts')
        glucose_value = float(event.get('value'))
        meal = exercise = insulin = 0

        meal_event = event.find('.//meal')
        if meal_event is not None:
            meal = float(meal_event.get('carbs'))

        exercise_event = event.find('.//exercise')
        if exercise_event is not None:
            exercise = float(exercise_event.get('intensity'))

        insulin_event = event.find('.//insulin')
        if insulin_event is not None:
            insulin = float(insulin_event.get('dose'))

        events.append((timestamp, glucose_value, meal, exercise, insulin))

    df = pd.DataFrame(events, columns=['timestamp', 'glucose_level', 'meal', 'exercise', 'insulin'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], format='%d-%m-%Y %H:%M:%S')
    return df

def preprocess_data(df):
    df.set_index('timestamp', inplace=True)
    df = df.sort_index()
    scaler = MinMaxScaler()
    df[['glucose_level', 'meal', 'exercise', 'insulin']] = scaler.fit_transform(df[['glucose_level', 'meal', 'exercise', 'insulin']])
    return df, scaler

def create_sequences(data, sequence_length=12):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length, :])
        y.append(data[i + sequence_length, 0])
    return np.array(X), np.array(y)
    

## 1.2 Model-specific Training

### 1.2.1 Random Forest with Cross-Validation

In [ ]:
def train_random_forest(X_train, y_train, X_val, y_val, patient, n_splits=5):
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    X_train_flat = X_train.reshape(X_train.shape[0], -1)

    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    train_rmse, val_rmse = [], []

    for train_index, val_index in kfold.split(X_train_flat):
        X_cv_train, X_cv_val = X_train_flat[train_index], X_train_flat[val_index]
        y_cv_train, y_cv_val = y_train[train_index], y_train[val_index]

        rf_model.fit(X_cv_train, y_cv_train)
        train_pred = rf_model.predict(X_cv_train)
        val_pred = rf_model.predict(X_cv_val)

        train_rmse.append(np.sqrt(mean_squared_error(y_cv_train, train_pred)))
        val_rmse.append(np.sqrt(mean_squared_error(y_cv_val, val_pred)))

    # Plot and save cross-validation RMSE
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, n_splits + 1), train_rmse, label='Training RMSE')
    plt.plot(range(1, n_splits + 1), val_rmse, label='Validation RMSE')
    plt.title(f"Random Forest Cross-Validation Training and Validation RMSE for {patient}")
    plt.xlabel("Fold")
    plt.ylabel("RMSE")
    plt.legend()
    plt.grid(True)
    os.makedirs(f"/content/drive/MyDrive/ddls_final_project/figures/{patient}", exist_ok=True)
    plt.savefig(f"/content/drive/MyDrive/ddls_final_project/figures/{patient}/random_forest_loss.png")
    plt.show()

    rf_model.fit(X_train_flat, y_train)
    return rf_model


### 1.2.2 XGBoost

In [ ]:
def train_xgboost(X_train, y_train, X_val, y_val, patient):
    X_train_flat = X_train.reshape(X_train.shape[0], -1)
    X_val_flat = X_val.reshape(X_val.shape[0], -1)
    dtrain = xgb.DMatrix(X_train_flat, label=y_train)
    dval = xgb.DMatrix(X_val_flat, label=y_val)

    params = {'objective': 'reg:squarederror', 'learning_rate': 0.1, 'max_depth': 5, 'eval_metric': 'rmse'}
    evals_result = {}
    evals = [(dtrain, 'train'), (dval, 'validation')]
    xgb_model = xgb.train(params, dtrain, num_boost_round=100, evals=evals, early_stopping_rounds=10, evals_result=evals_result, verbose_eval=False)

    # Plot and save XGBoost loss
    plt.figure(figsize=(10, 5))
    plt.plot(evals_result['train']['rmse'], label='Training RMSE')
    plt.plot(evals_result['validation']['rmse'], label='Validation RMSE')
    plt.title(f"XGBoost Training and Validation RMSE for {patient}")
    plt.xlabel("Epochs")
    plt.ylabel("RMSE")
    plt.legend()
    plt.grid(True)
    os.makedirs(f"/content/drive/MyDrive/ddls_final_project/figures/{patient}", exist_ok=True)
    plt.savefig(f"/content/drive/MyDrive/ddls_final_project/figures/{patient}/xgboost_loss.png")
    plt.show()

    return xgb_model
    

### 1.2.3 LSTM

In [ ]:
def build_lstm(input_shape):
    model = Sequential([
        Bidirectional(LSTM(128, return_sequences=True), input_shape=input_shape),
        Dropout(0.2),
        Bidirectional(LSTM(64, return_sequences=True)),
        Dropout(0.2),
        LSTM(32),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def train_lstm(X_train, y_train, X_val, y_val, input_shape, patient):
    model = build_lstm(input_shape)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping], verbose=0)

    # Plot and save LSTM loss
    plt.figure(figsize=(10, 5))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f"LSTM Training and Validation Loss for {patient}")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    os.makedirs(f"/content/drive/MyDrive/ddls_final_project/figures/{patient}", exist_ok=True)
    plt.savefig(f"/content/drive/MyDrive/ddls_final_project/figures/{patient}/lstm_loss.png")
    plt.show()

    return model
    

### 1.2.4 CNN-LSTM

In [ ]:
def build_cnn_lstm(input_shape):
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        LSTM(50, return_sequences=True),
        Dropout(0.2),
        LSTM(50),
        Dense(25, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def train_cnn_lstm(X_train, y_train, X_val, y_val, input_shape, patient):
    model = build_cnn_lstm(input_shape)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping], verbose=0)

    # Plot and save CNN-LSTM loss
    plt.figure(figsize=(10, 5))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f"CNN-LSTM Training and Validation Loss for {patient}")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    os.makedirs(f"/content/drive/MyDrive/ddls_final_project/figures/{patient}", exist_ok=True)
    plt.savefig(f"/content/drive/MyDrive/ddls_final_project/figures/{patient}/cnn_lstm_loss.png")
    plt.show()

    return model
    

## 1.3 Model Evaluation with Visualizations

In [ ]:
def evaluate_model(model, X_test, y_test, model_type, scaler, patient, df_train, df_test):
    # Flatten data if required by model
    if model_type in ['random_forest', 'xgboost']:
        X_test = X_test.reshape(X_test.shape[0], -1)

    if model_type == 'xgboost':
        dtest = xgb.DMatrix(X_test)
        y_pred = model.predict(dtest)
    else:
        y_pred = model.predict(X_test).flatten()

    # Rescale predictions and ground truth back to original scale
    y_pred_rescaled = scaler.inverse_transform(np.concatenate([y_pred.reshape(-1, 1), np.zeros((len(y_pred), 3))], axis=1))[:, 0]
    y_test_rescaled = scaler.inverse_transform(np.concatenate([y_test.reshape(-1, 1), np.zeros((len(y_test), 3))], axis=1))[:, 0]

    # 1. Plot Predicted vs. Ground Truth for All Testing Data
    plt.figure(figsize=(12, 6))
    test_times = df_test.index[-len(y_test_rescaled):]
    plt.plot(test_times, y_test_rescaled, label='True Glucose Levels', color='blue')
    plt.plot(test_times, y_pred_rescaled, label='Predicted Glucose Levels', linestyle='--', color='orange')
    plt.xlabel('Time')
    plt.ylabel('Glucose Level (mg/dL)')
    plt.title(f"Predicted vs Actual Glucose Levels for {patient} ({model_type}) - Full Testing Data")
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    os.makedirs(f"/content/drive/MyDrive/ddls_final_project/figures/{patient}", exist_ok=True)
    plt.savefig(f"/content/drive/MyDrive/ddls_final_project/figures/{patient}/{model_type}_prediction_full.png")
    plt.show()

    # 2. Plot Past 7 Days (or available data) and Next 48 Hours
    # Ensure the training data is sorted by timestamp to retrieve the last records accurately
    df_train = df_train.sort_index()

    # Get the last timestamp in the training data
    last_train_timestamp = df_train.index[-1]

    # Retrieve up to 7 days of data before the last timestamp
    last_7_days_data = df_train[df_train.index >= last_train_timestamp - pd.Timedelta(days=7)]

    # Extract glucose_level column and reshape for inverse transformation
    glucose_levels_scaled = last_7_days_data[['glucose_level', 'meal', 'exercise', 'insulin']].values
    glucose_levels_rescaled = scaler.inverse_transform(glucose_levels_scaled)[:, 0]  # Only use the glucose_level column

    # Define thresholds for color coding
    low_threshold = 70
    high_threshold = 180

    # Calculate the maximum glucose level for dynamic y-limit
    max_glucose_level_train = np.max(glucose_levels_rescaled)  # Get max from training data
    max_glucose_level_pred = y_pred_rescaled[:48 * 12].max()   # Get max from the first 48 hours of predictions
    max_glucose_level = max(max_glucose_level_train, max_glucose_level_pred)

    # Set ylim based on the max glucose level
    if max_glucose_level <= 350:
        ylim_upper = 350
    elif max_glucose_level <= 400:
        ylim_upper = 400
    else:
        ylim_upper = 450

    # Plot recorded glucose levels (last 7 days) and predictions (next 48 hours)
    plt.figure(figsize=(12, 6))
    plt.plot(last_7_days_data.index, glucose_levels_rescaled, label='Recorded Glucose Levels (Last 7 Days)', color='#BFB7A8')

    # Prepare time range for the next 48 hours predictions
    prediction_times = pd.date_range(start=last_train_timestamp + pd.Timedelta(minutes=5), periods=48 * 12, freq='5T')  # 48 hours, 5-minute intervals
    prediction_48h = y_pred_rescaled[:48 * 12]  # Limit predictions to the first 48 hours

    # Plot color-coded predictions
    plt.plot([], [], color='purple', linewidth=2.5, label='Hypoglycemia (< 70 mg/dL)')
    plt.plot([], [], color='red', linewidth=2.5, label='Hyperglycemia (> 180 mg/dL)')
    plt.plot([], [], color='blue', linewidth=1.5, label='Normal Range')

    for i in range(len(prediction_times) - 1):
        glucose_level = prediction_48h[i]
        if glucose_level < low_threshold:
            plt.plot(prediction_times[i:i + 2], prediction_48h[i:i + 2], color='purple', linewidth=2.5)
        elif glucose_level > high_threshold:
            plt.plot(prediction_times[i:i + 2], prediction_48h[i:i + 2], color='red', linewidth=2.5)
        else:
            plt.plot(prediction_times[i:i + 2], prediction_48h[i:i + 2], color='blue', linewidth=1.5)

    plt.xlabel('Time')
    plt.ylabel('Glucose Level (mg/dL)')
    plt.ylim(0, ylim_upper)
    plt.title(f"Recorded and Predicted Glucose Levels for {patient} ({model_type}) - Last 7 Days and Next 48 Hours")
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.gca().xaxis.set_major_locator(mdates.DayLocator())
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.savefig(f"/content/drive/MyDrive/ddls_final_project/figures/{patient}/{model_type}_prediction_48h.png")
    plt.show()

    # Calculate and return metrics
    rmse = np.sqrt(mean_squared_error(y_test_rescaled, y_pred_rescaled))
    mae = mean_absolute_error(y_test_rescaled, y_pred_rescaled)

    return rmse, mae
    

# 2. Train and Evaluate Models for Each Patient

In [ ]:
patients = {
    'patient559': {'train': '/content/drive/MyDrive/ddls_final_project/data/559-ws-training.xml', 'test': '/content/drive/MyDrive/ddls_final_project/data/559-ws-testing.xml'},
    'patient563': {'train': '/content/drive/MyDrive/ddls_final_project/data/563-ws-training.xml', 'test': '/content/drive/MyDrive/ddls_final_project/data/563-ws-testing.xml'},
    'patient570': {'train': '/content/drive/MyDrive/ddls_final_project/data/570-ws-training.xml', 'test': '/content/drive/MyDrive/ddls_final_project/data/570-ws-testing.xml'},
    'patient575': {'train': '/content/drive/MyDrive/ddls_final_project/data/575-ws-training.xml', 'test': '/content/drive/MyDrive/ddls_final_project/data/575-ws-testing.xml'},
    'patient588': {'train': '/content/drive/MyDrive/ddls_final_project/data/588-ws-training.xml', 'test': '/content/drive/MyDrive/ddls_final_project/data/588-ws-testing.xml'},
    'patient591': {'train': '/content/drive/MyDrive/ddls_final_project/data/591-ws-training.xml', 'test': '/content/drive/MyDrive/ddls_final_project/data/591-ws-testing.xml'}
}

results = []

for patient, files in patients.items():
    # Load and preprocess training data
    df_train = load_xml_data(files['train'])
    df_train, scaler = preprocess_data(df_train)
    X, y = create_sequences(df_train.values)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Load and preprocess testing data
    df_test = load_xml_data(files['test'])
    df_test.set_index('timestamp', inplace=True)
    df_test[['glucose_level', 'meal', 'exercise', 'insulin']] = scaler.transform(df_test[['glucose_level', 'meal', 'exercise', 'insulin']])
    X_test, y_test = create_sequences(df_test.values)

    # Random Forest
    rf_model = train_random_forest(X_train, y_train, X_val, y_val, patient)
    rmse, mae = evaluate_model(rf_model, X_test, y_test, 'random_forest', scaler, patient, df_train, df_test)
    results.append([patient, 'Random Forest', rmse, mae])

    # XGBoost
    xgb_model = train_xgboost(X_train, y_train, X_val, y_val, patient)
    rmse, mae = evaluate_model(xgb_model, X_test, y_test, 'xgboost', scaler, patient, df_train, df_test)
    results.append([patient, 'XGBoost', rmse, mae])

    # LSTM
    input_shape = (X_train.shape[1], X_train.shape[2])
    lstm_model = train_lstm(X_train, y_train, X_val, y_val, input_shape, patient)
    rmse, mae = evaluate_model(lstm_model, X_test, y_test, 'lstm', scaler, patient, df_train, df_test)
    results.append([patient, 'LSTM', rmse, mae])

    # CNN-LSTM
    cnn_lstm_model = train_cnn_lstm(X_train, y_train, X_val, y_val, input_shape, patient)
    rmse, mae = evaluate_model(cnn_lstm_model, X_test, y_test, 'cnn-lstm', scaler, patient, df_train, df_test)
    results.append([patient, 'CNN-LSTM', rmse, mae])

    # Save models
    os.makedirs(f"/content/drive/MyDrive/ddls_final_project/models/{patient}", exist_ok=True)
    joblib.dump(rf_model, f"/content/drive/MyDrive/ddls_final_project/models/{patient}/random_forest_model.pkl")
    xgb_model.save_model(f"/content/drive/MyDrive/ddls_final_project/models/{patient}/xgboost_model.json")
    lstm_model.save(f"/content/drive/MyDrive/ddls_final_project/models/{patient}/lstm_model.h5")
    cnn_lstm_model.save(f"/content/drive/MyDrive/ddls_final_project/models/{patient}/cnn_lstm_model.h5")

results_df = pd.DataFrame(results, columns=['Patient', 'Model', 'RMSE', 'MAE'])
results_df